Do before running:
Manually updat paths hard-coded in the json files in /Users/osorio/notebooks/od_calibration_sumo_files/spsa_sumo_platform-main/

Currently the simulation seed is not varied, and a single simulation replication is run for each OD. 

# Imports

In [1]:
import os
#%%
import pandas as pd
import numpy as np
import subprocess
import os, sys, json
from typing import Union
from pathlib import Path

classes/code below are  from TUM group, see links below, cite appropriately when publishing work
https://github.com/LastStriker11/spsa_sumo_platform

In [2]:
 os.chdir("od_calibration_sumo_files") #/spsa_sumo_platform-main/") 

# Main functions

In [3]:
# [TUM] miscs functions
def load_experiment_config(config: Union[Path, str]="config.json",
                           sim_setup: Union[Path, str]="simulation_setups.json"):
    """Load paths, simulation setups and algorithm setups
    

    Parameters
    ----------
    config : Union[Path, str], optional
        Paths to cache, network, and SUMO. The default is "config.json".
    sim_setup : Union[Path, str], optional
        Simulation parameters. The default is "simulation_setups.json".

    Returns
    -------
    config : Dictionary
    sim_setup : Dictionary

    """
    config = Path(config) if isinstance(config, str) else config
    config = json.load(open(config))
    for k, v in config.items():
        config[k] = Path(v)
    
    sim_setup = Path(sim_setup) if isinstance(sim_setup, str) else sim_setup
    sim_setup = json.load(open(sim_setup))
    for k, v in sim_setup.items():
        sim_setup[k] = v
            
    return config, sim_setup

config, sim_setup = load_experiment_config()


In [4]:
def parse_loop_data(config, loop_file):
    """Read the Loop Detectors Data: Each SUMO run produces a file with the
    traffic counts. This function reads the corresponding traffic counts file
    averages across simulation replications
    """        
    output_file =(config["NETWORK"] / "loopOutputs.csv")
    data2csv = (
        f"python {config['SUMO']}/tools/xml/xml2csv.py "
        f"{config['NETWORK']/loop_file} "
        f"--x {config['SUMO']}/data/xsd/det_e1_file.xsd " 
        f"-o {output_file}"
        )
    
    os.system(data2csv)
    
    df_trips = pd.read_csv(output_file, sep=";", header=0)
    df_trips["EdgeID"] = [elem.split("_")[1] for elem in df_trips["interval_id"]]
    df_trips.groupby(by=['EdgeID','interval_begin','interval_end'], as_index=False).agg({'interval_nVehContrib':np.sum, 'interval_harmonicMeanSpeed':np.mean})
    return df_trips
    

In [5]:
# [TUM] run_sumo functions
def run_sumo(config, sim_setup, algo_iter, replic_iter):
    # Run OD2Trips and SUMO simulation.

    # p_reroute = 0.1 # rerouting probability
    # Run od2trips_cmd to generate trips file
    od2trips_cmd = (
        f"od2trips --no-step-log  --spread.uniform "
        f"--taz-files {config['NETWORK']}/{sim_setup['taz']} "
        f"--tazrelation-files {config['NETWORK']}/iter{algo_iter}_replic{replic_iter}_{sim_setup['current_od']} "
        f"-o {config['NETWORK']}/iter{algo_iter}_replic{replic_iter}_od_trips.trips.xml "
        )
        # f"--seed {seed}"
        # --output-prefix {k}
    print(od2trips_cmd)
    os.system(od2trips_cmd)
    
    # Run SUMO to generate outputs
    sumo_run = (
        f"sumo --output-prefix iter{algo_iter}_replic{replic_iter}_ "
        f"--ignore-route-errors=true "
        f"--net-file={config['NETWORK']/sim_setup['net']} "
        f"--routes={config['NETWORK']}/iter{algo_iter}_replic{replic_iter}_od_trips.trips.xml "
        f"-b {sim_setup['start_sim_sec']} -e {sim_setup['end_sim_sec']} "
        f"--additional-files {config['NETWORK']/sim_setup['add']} "
        f"--duration-log.statistics "
        f"--xml-validation never "
        f"--vehroutes {config['NETWORK']}/routes.vehroutes.xml "
        )
        # f"--seed {seed}"
    print(sumo_run)
    os.system(sumo_run)

In [6]:
def get_gt_data(config, sim_setup): 
    """Read and parse SUMO outputs.
    
    """    
    # currently defined for a single simulation replication
    
    #np.random.seed(11)
    # random_seeds = np.random.normal(0, 10000, sim_setup["n_sumo_replicate"]).astype("int32")

    # Run od2trips_cmd to generate trips file
    od2trips_cmd = (
        f"od2trips --no-step-log  --spread.uniform "
        f"--taz-files {config['NETWORK']}/{sim_setup['taz']} "
        f"--tazrelation-files {config['NETWORK']}/{sim_setup['gt_od']} "
        f"-o {config['NETWORK']}/gt_od_trips.trips.xml "
        )
        # f"--seed {seed}"
        # --output-prefix {k}
    print(od2trips_cmd)
    os.system(od2trips_cmd)
    
    # Run SUMO to generate outputs
    sumo_run = (
        f"sumo --output-prefix GT "
        f"--ignore-route-errors=true "
        f"--net-file={config['NETWORK']/sim_setup['net']} "
        f"--routes={config['NETWORK']}/gt_od_trips.trips.xml "
        f"-b {sim_setup['start_sim_sec']} -e {sim_setup['end_sim_sec']} "
        f"--additional-files {config['NETWORK']/sim_setup['add']} "
        f"--duration-log.statistics "
        f"--xml-validation never "
        f"--vehroutes {config['NETWORK']}/gt_routes.vehroutes.xml "
        )
        # f"--seed {seed}"
    print(sumo_run)
    os.system(sumo_run)
    

    if sim_setup["objective"] == "counts":
        loop_file = "GTout.xml"
        df1 = parse_loop_data(config, loop_file)

        return df1.groupby(by=['EdgeID','interval_begin','interval_end'], as_index=False).agg(
            {'interval_nVehContrib':np.sum, 'interval_harmonicMeanSpeed':np.mean})
    # speeds across lanes are being averaged arithmetically not with harmonic mean


In [7]:
def compute_nrmse_counts(df_true, df_simulated):
    df1 = df_true.merge(df_simulated, on=['EdgeID', 'interval_begin', 'interval_end'],  suffixes=('_GT', '_sim'), how='left')
    df1['diff_square'] = (df1['interval_nVehContrib_GT'] - df1['interval_nVehContrib_sim'])**2
    n = df1.shape[0]
    RMSN = np.sqrt(n*(df1['diff_square'].sum()))/df1['interval_nVehContrib_GT'].sum()
    return RMSN

In [8]:
def run_and_parse_sumo_outputs(config, sim_setup, algo_iter): 
    """Read and parse SUMO outputs.
    
    """    
    #np.random.seed(11)
    # random_seeds = np.random.normal(0, 10000, sim_setup["n_sumo_replicate"]).astype("int32")
    for replic_counter in range(sim_setup["n_sumo_replicate"]):
        # Run simulation
        run_sumo(config, sim_setup, algo_iter, replic_counter) #, random_seeds[counter])

        if sim_setup["objective"] == "counts":
            loop_file = f"iter{algo_iter}_replic{replic_counter}_out.xml"
            df_agg = parse_loop_data(config, loop_file)

            if replic_counter==0:
                df_loop = df_agg.copy()
            else:
                df_loop = pd.concat([df_loop, df_agg], axis=0)
                

    if sim_setup["objective"] == "counts":
        return df_loop.groupby(by=['EdgeID','interval_begin','interval_end'], as_index=False).agg(
            {'interval_nVehContrib':np.mean, 'interval_harmonicMeanSpeed':np.mean})
    # speeds across lanes are being averaged arithmetically not with harmonic mean


In [9]:
def generate_od_xml(od_rand, config, setup, algo_iter, replic_iter):
    # print od in xml file
    

    od_xml_str = (
    f"<?xml version=\"1.0\" encoding=\"UTF-8\"?> \n"
    f"<data xmlns:xsi=\"http://www.w3.org/2001/XMLSchema-instance\" xsi:noNamespaceSchemaLocation=\"http://sumo.dlr.de/xsd/datamode_file.xsd\">\n"
    f"<interval id=\"CarA\" begin=\"54000\" end=\"57600.0\">\n"
    f"<tazRelation from=\"taz91\" to=\"taz93\" count=\"{int(np.floor(od_rand[0]))}\"/>\n"
    f"<tazRelation from=\"taz91\" to=\"taz94\" count=\"{int(np.floor(od_rand[1]))}\"/>\n"
    f"<tazRelation from=\"taz92\" to=\"taz93\" count=\"{int(np.floor(od_rand[2]))}\"/>\n"
    f"<tazRelation from=\"taz92\" to=\"taz94\" count=\"{int(np.floor(od_rand[3]))}\"/>\n"
    f"</interval>\n"
    f"</data>")

    file_name = f"{config['NETWORK']}/iter{algo_iter}_replic{replic_iter}_{sim_setup['current_od']}"
    print('printing '+file_name)
    f = open(file_name,'w')
    f.write(od_xml_str) 
    f.close()    

If the execution of od2trips_cmd or sumo_run do not work and you get an error along the lines of 
'FileNotFoundError: [Errno 2] No such file or directory: 'sumo --output-prefix 1 ...'
then troubleshoot by running sumo from a terminal + command line

In [10]:
# uncomment below to test running sumo
# run_sumo(config, sim_setup, 4)

# Generate ground truth (GT) OD and GT counts and speeds data
GT OD is defined in quickstart.gt_od.xml:

        <tazRelation from="taz91" to="taz93" count="500"/>
        <tazRelation from="taz91" to="taz94" count="500"/>
        <tazRelation from="taz92" to="taz93" count="900"/>
        <tazRelation from="taz92" to="taz94" count="900"/>
        
corresponds to GT counts:

 	EdgeID 	interval_begin 	interval_end 	interval_nVehContrib 	interval_harmonicMeanSpeed
    0 	#EdgeL5 	54000.0 	57600.0 	959.0 	13.500000
    1 	#EdgeL7 	54000.0 	57600.0 	1243.0 	13.603333



In [11]:
df_gt_data = get_gt_data(config, sim_setup)

od2trips --no-step-log  --spread.uniform --taz-files /Users/osorio/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.taz.xml --tazrelation-files /Users/osorio/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.gt_od.xml -o /Users/osorio/notebooks/od_calibration_sumo_files/quickstart/data/gt_od_trips.trips.xml 
Success.
sumo --output-prefix GT --ignore-route-errors=true --net-file=/Users/osorio/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.net.xml --routes=/Users/osorio/notebooks/od_calibration_sumo_files/quickstart/data/gt_od_trips.trips.xml -b 54000 -e 57600 --additional-files /Users/osorio/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.additional.xml --duration-log.statistics --xml-validation never --vehroutes /Users/osorio/notebooks/od_calibration_sumo_files/quickstart/data/gt_routes.vehroutes.xml 
Loading net-file from '/Users/osorio/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.net.xml' ... done (6ms).
Load

In [12]:
df_gt_data

,EdgeID,interval_begin,interval_end,interval_nVehContrib,interval_harmonicMeanSpeed
0,#EdgeL5,54000.0,57600.0,959,13.500000
1,#EdgeL7,54000.0,57600.0,1243,13.603333


# Calibrate

In [13]:
num_calib_iterations = 5

In [14]:
for iter1 in range(num_calib_iterations):
    
    # define new OD to simulate
    # replace below random sampling with BO step that yields an OD to simulate for this iteration
    # BO problem: use lower bounds of zero, upper bounds of 2000
    
    od_rand = np.random.rand(4)*200
    print(iter1, np.sum(od_rand))
    
    # print new OD into xml file to be used as sumo input
    # assumes a single simulation replication is run
    generate_od_xml(od_rand, config, sim_setup, iter1, 0)
    
    df_simulated = run_and_parse_sumo_outputs(config, sim_setup, iter1)
    
    # compute nrmse
    loss_estimate = compute_nrmse_counts(df_gt_data, df_simulated)
    print('loss estimate: ',loss_estimate)



0 315.838973726413
printing /Users/osorio/notebooks/od_calibration_sumo_files/quickstart/data/iter0_replic0_quickstart.current_od.xml
od2trips --no-step-log  --spread.uniform --taz-files /Users/osorio/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.taz.xml --tazrelation-files /Users/osorio/notebooks/od_calibration_sumo_files/quickstart/data/iter0_replic0_quickstart.current_od.xml -o /Users/osorio/notebooks/od_calibration_sumo_files/quickstart/data/iter0_replic0_od_trips.trips.xml 
Success.
sumo --output-prefix iter0_replic0_ --ignore-route-errors=true --net-file=/Users/osorio/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.net.xml --routes=/Users/osorio/notebooks/od_calibration_sumo_files/quickstart/data/iter0_replic0_od_trips.trips.xml -b 54000 -e 57600 --additional-files /Users/osorio/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.additional.xml --duration-log.statistics --xml-validation never --vehroutes /Users/osorio/notebooks/od_calibr